In [ ]:
import os
import json
os.chdir('C:\\Users\\Romina\\Desktop\\thesis')

import datetime as dt
from datetime import datetime, date
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

In [ ]:
os.getcwd()

In [ ]:
data=pd.read_csv("final_session_30mins.csv", low_memory=False)
labels=pd.read_csv("sessionswithLables.csv", low_memory=False)
session_withlabel=pd.merge(data, labels[['session_no', 'distance_0']], right_on='session_no', left_on='session_no', how='left')
session_withlabel.to_csv("session_withlabel.csv")

In [ ]:
data.info()
data.head()

In [ ]:
# from urllib.parse import urlparse
# #data['http_referer_']=data['http_referer'].apply(lambda x: str(x.encode('utf-16'))


# data['refere_domain']=data['http_referer'].apply(lambda x: urlparse(x).netloc)

m = data['http_referer'].str.extract('(?<=http://)(.*?)(?=/)|(?<=https://)(.*?)(?=/)')
m = m[0].fillna(m[1]).fillna(data['http_referer'])

data['DOMAINNAME'] = m



In [ ]:
import numpy as n
data['DOMAINNAME_g'] = np.where(data['DOMAINNAME'].str.contains("google|yahoo|bing|duckduckgo|ask", na=False), "Search_engine", data['DOMAINNAME']) 
data['DOMAINNAME_g'] = np.where(data['DOMAINNAME_g'].str.contains(".edu|uni|lib|csiro", na=False), "Education",data['DOMAINNAME_g']) 
data['DOMAINNAME_g'] = np.where(data['DOMAINNAME_g'] .str.contains(".gov", na=False), "Government",data['DOMAINNAME_g']) 
data['DOMAINNAME_g'] = np.where(data['DOMAINNAME_g'] .str.contains("researchdata.ands.|www.ands.", na=False), "ARDC",data['DOMAINNAME_g']) 
data['DOMAINNAME_g'] = np.where(data['DOMAINNAME_g'] .str.contains("wikipedia.", na=False), "Wikipeida",data['DOMAINNAME_g']) 


In [ ]:
len(data['DOMAINNAME'].unique())

In [ ]:
a=data[['DOMAINNAME_g']]
a=a.drop_duplicates()
len(a)
#a.to_csv('domains.csv')

In [ ]:
#data.head(30)

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
#data.head(30)
# data['Date'] = data['timestamp'].dt.date
# data['Time'] = data['timestamp'].dt.time

In [ ]:
data['Date']= data['timestamp'].dt.date
data['time']= data['timestamp'].dt.strftime('%H:%M:%S')

In [ ]:
data['Advance_search']=data['q'].str.contains("AND" or "OR")


In [ ]:
def data_processor(dat):
    dat=dat.apply(lambda x: str(x).replace(r"[OR, AND]", " "))
    #dat=dat.apply(lambda x: str(x).lower())
    #dat=dat.apply(lambda x: x.replace('\d+', ''))
    dat=dat.apply(lambda x: str(x).replace('[^\w\s]',""))
    dat=dat.apply(lambda x: str(x).replace(r"[︰-＠]", ""))
    dat=dat.apply(lambda x: str(x).replace(r"[_text_:]", ""))
    return dat

In [ ]:
data['q']=data_processor(data['q'])
data['q']=data['q'].where(~data['q'].str.isdigit())
data["q"]=data["q"].replace('nan', np.NaN)
data.info()
data.head(50)

In [ ]:
#grab all the sessions with atleast one search
grouped = data.groupby('session_no')['ip','q'].count().reset_index()

#non_abandon=grouped[(grouped['_source.user.ip'] > 1) & (grouped['_source.doc.@fields.filters.q'] > 0)]
sessWithsearch=grouped[(grouped['q'] > 0)] #SESSIONS With atleast a search Query
#sessWithsearch.head()

sessWithsearch_dat=data[data['session_no'].isin(sessWithsearch['session_no'])].reset_index()
sessWithsearch_dat=sessWithsearch_dat.drop(['Unnamed: 0','Unnamed: 0.1' ], axis=1)
sessWithsearch_dat.head(30)

In [ ]:
#sessWithsearch_dat.to_csv('test.csv')

In [ ]:
# b=sessWithsearch_dat[sessWithsearch_dat['session_no']==495807]
# b.to_csv("test2.csv")

In [ ]:
#flag sessions with views (click) and those accessed portal (Go To Data Provider, Go To Service Provider))
grouped_event = sessWithsearch_dat.groupby(['session_no','event'])['ip'].count().reset_index()
grouped_event.head()

#pivot and append to the entire data
grouped_event_=grouped_event.pivot(index='session_no', columns='event', values='ip')
grouped_event_=grouped_event_[['portal_search','portal_view', 'portal_accessed', 'portal_export']].reset_index()
grouped_event_.head()

grouped_event_=grouped_event_.fillna(0)
grouped_event_.head(30)

In [ ]:
import datetime

#sessWithsearch_dat['Date']= sessWithsearch_dat['Date'].
sessWithsearch_dat['time']= pd.to_datetime(sessWithsearch_dat['time'],format= '%H:%M:%S' ).dt.time
sessWithsearch_dat.info()


In [ ]:
#flag sessions with views (click) and those accessed portal (Go To Data Provider, Go To Service Provider))
grouped_mintime=sessWithsearch_dat.sort_values("time").groupby("session_no", as_index=False).first()
grouped_mintime=grouped_mintime[['session_no', 'DOMAINNAME_g', 'time']]
grouped_mintime.head()
#grouped_mintime.info()

In [ ]:
grouped_maxtime=sessWithsearch_dat.groupby(['session_no'])['time'].max().reset_index().rename(columns={'time':'time_finish'})
grouped_maxtime
# grouped_maxtime.info()
grouped_maxtime.tail()

In [ ]:
#number of queries per session
grouped_quer = sessWithsearch_dat.groupby(['session_no'])['q'].count().reset_index()
#grouped_quer 

In [ ]:
all_words=sessWithsearch_dat[['session_no', 'q']]
all_words=all_words.dropna(subset=['q'])
all_words.head(30)
all_words['q']=all_words['q'].apply(lambda x: x.split())

In [ ]:
all_words.head()

In [ ]:
group_words=all_words.groupby('session_no').sum().reset_index()
group_words.info()

group_words.head()

In [ ]:
print(group_words)

In [ ]:
group_words['q_']=group_words['q'].apply(lambda x: ' '.join(str(w) for w in x))
# group_words=group_words['q'].apply(lambda x:x.replace('[]', ''))
#group_words['q_']= group_words['q_'].apply(lambda x: x.replace("_text_:", '' ))
#print(group_words)

In [ ]:
#group_words.info()

In [ ]:
group_words['q_']=group_words['q_'].apply(lambda x: x.lower())
group_words['no_vocab']=group_words['q_'].apply(lambda x: len(set(x.split())))
group_words['no_word']=group_words['q_'].apply(lambda x: len(x.split()))

In [ ]:
group_words.head()
group_words.info()
group_words['no_vocab'].max()
a=group_words[group_words['no_vocab']>200]
# group_words['no_vocab'].mean()
# group_words['no_vocab'].std()

In [ ]:
data_1=pd.merge(grouped_event_, grouped_mintime[['session_no', 'DOMAINNAME_g', 'time']],right_on='session_no', left_on='session_no', 
                how='left')


In [ ]:
data_1.info()
data_1.head()

In [ ]:
data_2=pd.merge(data_1, grouped_maxtime[['session_no', 'time_finish']],right_on='session_no', left_on='session_no', 
                how='left')
data_2.info()

In [ ]:
data_2=pd.merge(data_1, grouped_maxtime[['session_no', 'time_finish']],right_on='session_no', left_on='session_no', 
                how='left')
data_2.info()

In [ ]:
data_3=pd.merge(data_2, grouped_quer[['session_no', 'q']],right_on='session_no', left_on='session_no', 
                how='left')
data_3.info()


In [ ]:
data_4=pd.merge(data_3, group_words[['session_no', 'no_vocab', 'no_word']],right_on='session_no', left_on='session_no', 
                how='left')
data_4.info()


In [ ]:
#data_4.head(50)
#data_4['portal_accessed'].value_counts()
#data_4['portal_export'].value_counts()

In [ ]:
data_4['DOMAINNAME_g'].value_counts()

In [ ]:
data_4['session_type']=np.where((data_4['DOMAINNAME_g']!='Search_engine') & (data_4['DOMAINNAME_g']!="ARDC"),
#                                 & (data_4['DOMAINNAME_g']!='Education')& (data_4['DOMAINNAME_g']!='Government'),
                                "Browse_other", data_4['DOMAINNAME_g'])
data_4['session_type'].value_counts()

In [ ]:
#data_4['session_type'].value_counts()
from datetime import timedelta, date
data_4['time']= pd.to_timedelta(data_4['time'].astype(str))
data_4['time_finish']=pd.to_timedelta(data_4['time_finish'].astype(str))

data_4['session_len']= data_4['time_finish'] - data_4['time']
#datetime.combine(date.today(), data_4['time_finish']) - datetime.combine(date.today(), data_4['time'])

In [ ]:
dummy=pd.get_dummies(data_4['session_type'])
dummy.info()

In [ ]:
data_4['session_len_s']=data_4['session_len'].dt.seconds/60

In [ ]:
data_4[['session_len', 'session_len_s']]
data_4.info()

In [ ]:
# data_4_= data_4[[#'portal_search', 
#                     'portal_view', 
#                 # 'portal_accessed', 'portal_export', 
#                  'q', 'no_vocab', 'no_word', 'session_len_s']]

data_5=pd.concat([data_4, dummy], axis=1)
data_5.info()
data_5.head()

In [ ]:
#existience of advance search in the session
grouped_advancesearch = sessWithsearch_dat.groupby(['session_no'])['Advance_search'].max().reset_index()
grouped_advancesearch.head()

#grouped_advancesearch[grouped_advancesearch['Advance_search']==True]

In [ ]:
data_6=pd.merge(data_5, grouped_advancesearch [['session_no', 'Advance_search']],right_on='session_no', left_on='session_no', 
                how='left')
data_6.info()

In [ ]:
data_6. to_csv("clusteringdata-session30.csv")


In [ ]:
data_6.describe()

In [ ]:
data_vis= data_5[[#'portal_search', 
                    'portal_view', 
                # 'portal_accessed', 'portal_export', 
                 'q', 'no_vocab', 'no_word', 'session_len_s']]

data_vis.info()

data_vis=data_vis[data_vis['portal_view']<1]
data_vis.info()

In [ ]:
#data_5_=data_5[data_5['portal_view']>0]
#data_5_.info()

data_vis[['session_len_s']].describe()

In [ ]:
# #Plot styling
import seaborn as sns; sns.set()  # for plot styling
%matplotlib inline
import matplotlib.pyplot as plt

# # plt.rcParams['figure.figsize'] = (16, 9)
# # plt.style.use('ggplot')

for i in range(2,5,1):
    plt.rcParams['figure.figsize'] = (9, 5)
    plt.style.use('ggplot')
    #Visualizing the data - displot
    plot_portal_search = sns.distplot(data_vis.iloc[:, i])
#plot_portal_view = sns.distplot(data_5["portal_view"])
    plt.xlabel(data_vis.columns[i])
    plt.show()

In [ ]:
for i in range(0,5,1):
    plt.figure(figsize=(10,8))
    #Visualizing the data - displot
    y=data_vis.columns[i]
    #print(y)
    sns.boxplot(x=data_vis[y])
    plt.ylabel(data_vis.columns[i])
    #plt.show()

In [ ]:
 from scipy.stats import pearsonr
corr1= pearsonr(data_5['q'], data_5['session_len_s'])
print(corr1)

In [ ]:
data_5=data_5[(data_5['no_vocab']<200)& (data_5['session_len_s']<600) & (data_5['no_word']<1000) &
              (data_5['q']<100)]
#data_5_=data_5_[data_5_['no_vocab']<20]
#data_5.info()
#data_5.to_csv("data_5.csv")

data_5['w_v_ratio']=data_5['no_vocab']/data_5['no_word']
data_5['q_len_ave']=data_5['no_word']/data_5['q']

data_55=data_5[[#'portal_search', 
                    'portal_view', 
              # 'portal_accessed', 'portal_export', 
                 'q', 
  #  'no_vocab', 'no_word', 
    'w_v_ratio','q_len_ave',
    'session_len_s'
    ,'ARDC','Browse_other','Search_engine'
]]
